In [1]:
import folium
import requests
import pandas as pd
import numpy as np 
import random
import json
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


Let's again assume that you are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent foursquare_agent, as shown below.

In [2]:
CLIENT_ID = '5M5MTCRHVZZ2BXVS5JQE22GGGAFXF2BUG3SEL0LZRMPHT3MB' # your Foursquare ID
CLIENT_SECRET = 'ZZQ13GXA2PCBDEANJ0FTW41WNHLKJOEY0OFVEKVI1UDUFVDZ' # your Foursquare Secret
ACCESS_TOKEN = 'PZXYPPFXQLLGT0ICJCP42HBCA52DKQCUGMOKVQ0ZRTULI2TW' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5M5MTCRHVZZ2BXVS5JQE22GGGAFXF2BUG3SEL0LZRMPHT3MB
CLIENT_SECRET:ZZQ13GXA2PCBDEANJ0FTW41WNHLKJOEY0OFVEKVI1UDUFVDZ


In [15]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'lat:{latitude}, long: {longitude}')

lat:40.7149555, long: -74.0153365


## 1. Search for a specific venue category

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

In [45]:
search_query = '*'
radius = 500
print(search_query + ' .... OK!')

* .... OK!


In [46]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5M5MTCRHVZZ2BXVS5JQE22GGGAFXF2BUG3SEL0LZRMPHT3MB&client_secret=ZZQ13GXA2PCBDEANJ0FTW41WNHLKJOEY0OFVEKVI1UDUFVDZ&ll=40.7149555,-74.0153365&oauth_token=PZXYPPFXQLLGT0ICJCP42HBCA52DKQCUGMOKVQ0ZRTULI2TW&v=20180604&query=*&radius=500&limit=30'

In [47]:
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-47-f23a461d33e0>:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.country,location.formattedAddress,venuePage.id,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,4efd1635f790570bcf88cbfd,Blue Smoke,"[{'id': '4bf58dd8d48988d1df931735', 'name': 'B...",v-1608008261,False,255 Vesey St,btwn West & North End Ave,40.714642,-74.015332,"[{'label': 'display', 'lat': 40.714641818092, ...",...,United States,"[255 Vesey St (btwn West & North End Ave), New...",55111630,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52e2f72a498e1de9bce745b0,El Vez,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1608008261,False,259 Vesey St,North End Avenue,40.714730,-74.015630,"[{'label': 'display', 'lat': 40.71473, 'lng': ...",...,United States,"[259 Vesey St (North End Avenue), New York, NY...",NaN,Battery Park City,292345,https://www.seamless.com/menu/el-vez-259-vesey...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png
2,4f22ca77e4b0ed3396a83a05,Conrad New York Downtown,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1608008261,False,102 N End Ave,NaN,40.715035,-74.015840,"[{'label': 'display', 'lat': 40.7150355, 'lng'...",...,United States,"[102 N End Ave, New York, NY 10282, United Sta...",NaN,Battery Park City,NaN,NaN,NaN,NaN,NaN,NaN
3,4dda977ad22d38ef4332c419,Shake Shack,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",v-1608008261,False,215 Murray St,btwn West St & North End Ave,40.715326,-74.014750,"[{'label': 'display', 'lat': 40.71532572032048...",...,United States,"[215 Murray St (btwn West St & North End Ave),...",NaN,NaN,726904,https://www.seamless.com/menu/shake-shack-215-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png
4,5accb33d2c7eb96824340d53,Seamore’s,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",v-1608008261,False,230 Vesey St,NaN,40.714242,-74.015085,"[{'label': 'display', 'lat': 40.71424164376645...",...,United States,"[230 Vesey St, New York, NY 10285, United States]",NaN,NaN,884804,https://www.seamless.com/menu/seamores-250-ves...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png


In [48]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Blue Smoke,BBQ Joint,255 Vesey St,btwn West & North End Ave,40.714642,-74.015332,"[{'label': 'display', 'lat': 40.714641818092, ...",34,10282,US,New York,NY,United States,"[255 Vesey St (btwn West & North End Ave), New...",NaN,4efd1635f790570bcf88cbfd
1,El Vez,Mexican Restaurant,259 Vesey St,North End Avenue,40.714730,-74.015630,"[{'label': 'display', 'lat': 40.71473, 'lng': ...",35,10282,US,New York,NY,United States,"[259 Vesey St (North End Avenue), New York, NY...",Battery Park City,52e2f72a498e1de9bce745b0
2,Conrad New York Downtown,Hotel,102 N End Ave,NaN,40.715035,-74.015840,"[{'label': 'display', 'lat': 40.7150355, 'lng'...",43,10282,US,New York,NY,United States,"[102 N End Ave, New York, NY 10282, United Sta...",Battery Park City,4f22ca77e4b0ed3396a83a05
3,Shake Shack,Burger Joint,215 Murray St,btwn West St & North End Ave,40.715326,-74.014750,"[{'label': 'display', 'lat': 40.71532572032048...",64,10282,US,New York,NY,United States,"[215 Murray St (btwn West St & North End Ave),...",NaN,4dda977ad22d38ef4332c419
4,Seamore’s,Seafood Restaurant,230 Vesey St,NaN,40.714242,-74.015085,"[{'label': 'display', 'lat': 40.71424164376645...",82,10285,US,New York,NY,United States,"[230 Vesey St, New York, NY 10285, United States]",NaN,5accb33d2c7eb96824340d53
5,Brookfield Place (BFPL),Shopping Mall,230 Vesey St,at West St,40.713240,-74.015193,"[{'label': 'display', 'lat': 40.71324018931094...",191,10281,US,New York,NY,United States,"[230 Vesey St (at West St), New York, NY 10281...",NaN,4a6fa0abf964a520a4d61fe3
6,Asphalt Green Battery Park City,Athletics & Sports,212 N End Ave,Murray Street,40.715924,-74.014673,"[{'label': 'display', 'lat': 40.71592364422336...",121,10282,US,New York,NY,United States,"[212 N End Ave (Murray Street), New York, NY 1...",NaN,51bcd723498eecb51dd996bc
7,3rd Floor Coffee Bar,Corporate Coffee Shop,200 West St,Murray St,40.714757,-74.014329,"[{'label': 'display', 'lat': 40.71475688776351...",87,10282,US,New York,NY,United States,"[200 West St (Murray St), New York, NY 10282, ...",NaN,58ece658424f93769dfd45d5
8,Battery Park City Esplanade,Park,Battery Park City,From Chambers St to Battery Park,40.711622,-74.017907,"[{'label': 'display', 'lat': 40.71162184551406...",429,10280,US,New York,NY,United States,[Battery Park City (From Chambers St to Batter...,NaN,4c1164576e5dc9b69506b02d
9,Del Frisco's Grille,Steakhouse,250 Vesey St,NaN,40.714004,-74.015012,"[{'label': 'display', 'lat': 40.7140038, 'lng'...",109,10281,US,New York,NY,United States,"[250 Vesey St, New York, NY 10281, United States]",Battery Park City,594301c89411f241f5b6a2ee


In [49]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=50) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [50]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.
